# 02 – Interaction Features
     Capturing Conditional Relationships Between Variables


## Objective

This notebook focuses on **interaction feature engineering**, covering:

- Why interactions matter beyond raw variables
- Business-driven vs automated interaction creation
- Numeric × numeric, numeric × categorical, and categorical × categorical interactions
- Risk-aware interaction design
- Leakage guardrails for interaction features

It answers:

    How do we capture conditional effects that single features cannot express?


## Why Interaction Features Matter

Many real-world effects are **conditional**, not additive.

Examples:
- High usage is good *unless* satisfaction is low
- Long tenure reduces churn *unless* support usage is high
- High income matters differently by customer segment

Linear models cannot learn these effects without interaction terms.



## Imports and Dataset



In [3]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns


df = pd.read_csv("../datasets/synthetic_customer_churn_classification_complete.csv")
df.head()


,customer_id,age,income,tenure_years,avg_monthly_usage,support_tickets_last_year,satisfaction_level,customer_segment,region,churn,future_retention_offer
0,1,18,NaN,2.012501,138.021163,1,NaN,segment_18,South,0,-0.069047
1,2,18,58991.061162,9.005550,213.043003,2,Very High,segment_98,West,0,-0.226607
2,3,67,31130.298545,3.633058,68.591582,2,Medium,segment_134,North,0,-0.065741
3,4,64,NaN,4.295957,28.790894,1,NaN,segment_72,North,0,0.061886
4,5,37,22301.231175,2.549855,100.136569,2,High,segment_147,East,1,1.073678


In [43]:
# Converting - satisfaction_level - into numeric variable
df["satisfaction_level"] = df["satisfaction_level"].map({'Very High':5, 'High':4, 'Medium':3, 'Low':2, 'Very Low' : 1, np.nan:3})

## Step 1 – Feature Overview


In [4]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   customer_id                10000 non-null  int64  
 1   age                        10000 non-null  int64  
 2   income                     7000 non-null   float64
 3   tenure_years               10000 non-null  float64
 4   avg_monthly_usage          9600 non-null   float64
 5   support_tickets_last_year  10000 non-null  int64  
 6   satisfaction_level         7500 non-null   object 
 7   customer_segment           10000 non-null  object 
 8   region                     10000 non-null  object 
 9   churn                      10000 non-null  int64  
 10  future_retention_offer     10000 non-null  float64
dtypes: float64(4), int64(4), object(3)
memory usage: 859.5+ KB


## Step 2 – Business Framing for Interactions

We focus on churn prediction.

Key hypotheses:
- Tenure moderates dissatisfaction risk
- Support volume amplifies dissatisfaction
- Usage value differs by customer segment
- Retention offers behave differently for at-risk customers


## Step 3 – Numeric × Numeric Interactions

Captures *magnitude amplification* effects.


In [44]:
# Enconding - Ordinal then execute operation of multiplying the avg_monthly_usage X satisfaction_level

df["usage_satisfaction_interaction"] = (
    df["avg_monthly_usage"].fillna(df["avg_monthly_usage"].median()) * df["satisfaction_level"]
)

df[["avg_monthly_usage", "satisfaction_level",
    "usage_satisfaction_interaction"]].head()


,avg_monthly_usage,satisfaction_level,usage_satisfaction_interaction
0,138.021163,3,414.063489
1,213.043003,5,1065.215014
2,68.591582,3,205.774746
3,28.790894,3,86.372681
4,100.136569,4,400.546277


## Tenure × Support Load

In [36]:
df["tenure_support_pressure"] = (
    df["tenure_years"] * df["support_tickets_last_year"]
)

df[["tenure_years", "support_tickets_last_year",
    "tenure_support_pressure"]].head()


,tenure_years,support_tickets_last_year,tenure_support_pressure
0,2.012501,1,2.012501
1,9.005550,2,18.011099
2,3.633058,2,7.266116
3,4.295957,1,4.295957
4,2.549855,2,5.099710


## Step 4 – Normalized Interaction Ratios

Ratios are often more stable than raw products.


In [37]:
df["support_tickets_per_year"] = (
    df["support_tickets_last_year"] / (df["tenure_years"] + 1e-3)
)

df[["support_tickets_last_year", "tenure_years",
    "support_tickets_per_year"]].head()


,support_tickets_last_year,tenure_years,support_tickets_per_year
0,1,2.012501,0.496647
1,2,9.005550,0.222061
2,2,3.633058,0.550349
3,1,4.295957,0.232723
4,2,2.549855,0.784051


## Step 5 – Numeric × Categorical Interactions

Conditional numeric effects by group.


In [39]:
df["customer_segment"].unique()

array(['segment_18', 'segment_98', 'segment_134', 'segment_72',
       'segment_147', 'segment_146', 'segment_5', 'segment_109',
       'segment_43', 'segment_28', 'segment_149', 'segment_48',
       'segment_14', 'segment_90', 'segment_120', 'segment_92',
       'segment_129', 'segment_46', 'segment_89', 'segment_51',
       'segment_3', 'segment_96', 'segment_83', 'segment_82',
       'segment_91', 'segment_33', 'segment_113', 'segment_127',
       'segment_110', 'segment_78', 'segment_128', 'segment_140',
       'segment_50', 'segment_69', 'segment_44', 'segment_26',
       'segment_105', 'segment_64', 'segment_2', 'segment_38',
       'segment_144', 'segment_70', 'segment_45', 'segment_37',
       'segment_17', 'segment_61', 'segment_123', 'segment_135',
       'segment_22', 'segment_10', 'segment_1', 'segment_25',
       'segment_74', 'segment_21', 'segment_71', 'segment_85',
       'segment_100', 'segment_23', 'segment_34', 'segment_11',
       'segment_119', 'segment_115', 'segm

In [38]:
for segment in df["customer_segment"].unique():
    df[f"usage_segment_{segment}"] = (
        df["avg_monthly_usage"] * (df["customer_segment"] == segment).astype(int)
    )

df.filter(like="usage_segment_").head()


C:\Users\pantu\AppData\Local\Temp\ipykernel_21292\3172586616.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"usage_segment_{segment}"] = (
C:\Users\pantu\AppData\Local\Temp\ipykernel_21292\3172586616.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"usage_segment_{segment}"] = (
C:\Users\pantu\AppData\Local\Temp\ipykernel_21292\3172586616.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all col

,usage_segment_segment_18,usage_segment_segment_98,usage_segment_segment_134,usage_segment_segment_72,usage_segment_segment_147,usage_segment_segment_146,usage_segment_segment_5,usage_segment_segment_109,usage_segment_segment_43,usage_segment_segment_28,...,usage_segment_segment_27,usage_segment_segment_76,usage_segment_segment_125,usage_segment_segment_53,usage_segment_segment_121,usage_segment_segment_77,usage_segment_segment_122,usage_segment_segment_55,usage_segment_segment_6,usage_segment_segment_99
0,138.021163,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,213.043003,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.000000,68.591582,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.000000,0.000000,28.790894,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.000000,0.000000,0.000000,100.136569,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Step 6 – Categorical × Categorical Interactions

Encodes joint group membership.

### Segment × Region Interaction

In [40]:
df["segment_region"] = (
    df["customer_segment"].astype(str) + "_" +
    df["region"].astype(str)
)

df["segment_region"].value_counts().head()


C:\Users\pantu\AppData\Local\Temp\ipykernel_21292\4184937022.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["segment_region"] = (


segment_region
segment_104_North    41
segment_41_North     40
segment_59_North     39
segment_144_North    38
segment_56_North     36
Name: count, dtype: int64

## Step 7 – Risk-Aware Binary Interaction Flags

Binary flags often outperform continuous interactions.


### High Support + Low Satisfaction

In [45]:
df["high_support_low_satisfaction"] = (
    (df["support_tickets_last_year"] > df["support_tickets_last_year"].median()) &
    (df["satisfaction_level"] < df["satisfaction_level"].median())
).astype(int)

df["high_support_low_satisfaction"].value_counts()


C:\Users\pantu\AppData\Local\Temp\ipykernel_21292\1171742323.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["high_support_low_satisfaction"] = (


high_support_low_satisfaction
0    9401
1     599
Name: count, dtype: int64

## Step 8 – Policy and Treatment Interaction

`future_retention_offer` is a **post-decision variable**.

Must NOT be used for churn prediction unless modeling uplift.


### Illustration Only (DO NOT USE FOR PREDICTION)

In [46]:
df["offer_satisfaction_interaction"] = (
    df["future_retention_offer"] * df["satisfaction_level"]
)


C:\Users\pantu\AppData\Local\Temp\ipykernel_21292\991344599.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["offer_satisfaction_interaction"] = (


## Step 9 – Leakage Guardrails

Interaction features must:
- Use only pre-outcome variables
- Avoid policy leakage
- Be computable at scoring time
- Be stable under deployment


## Step 10 – Signal Sanity Checks
### Correlation with Target

In [47]:
interaction_features = [
    "usage_satisfaction_interaction",
    "tenure_support_pressure",
    "support_tickets_per_year",
    "high_support_low_satisfaction"
]

df[interaction_features + ["churn"]].corr()


,usage_satisfaction_interaction,tenure_support_pressure,support_tickets_per_year,high_support_low_satisfaction,churn
usage_satisfaction_interaction,1.000000,0.007644,-0.012369,-0.127353,0.052185
tenure_support_pressure,0.007644,1.000000,-0.061155,0.174630,-0.108453
support_tickets_per_year,-0.012369,-0.061155,1.000000,0.029452,0.061687
high_support_low_satisfaction,-0.127353,0.174630,0.029452,1.000000,0.125909
churn,0.052185,-0.108453,0.061687,0.125909,1.000000


## Interpretability Check

Each interaction should answer:
- What two effects are being combined?
- Why should this matter to churn?
- Can this be explained to business stakeholders?


## Common Mistakes (Avoided)

- `[neg] - ` Blind polynomial expansion
- `[neg] - ` Exploding feature space
- `[neg] - ` Including post-outcome variables
- `[neg] - ` Creating uninterpretable interactions


## Summary Table

| Interaction Type | Example |
|----------------|--------|
| Numeric × Numeric | Usage × Satisfaction |
| Ratio | Support / Tenure |
| Numeric × Category | Usage × Segment |
| Category × Category | Segment × Region |
| Binary Flag | High support & low satisfaction |


## Key Takeaways

- Interactions encode conditional logic
- Business reasoning should guide design
- Ratios often outperform raw products
- Binary flags are powerful and stable
- Guard against leakage aggressively


## Next Notebook

03_Feature_Engineering/

└── [03_aggregation_and_window_features.ipynb](03_aggregation_and_window_features.ipynb)
